CS471 Final Project

In [1]:
print("hello")
import pandas as pd

hello


Read in the data, convert to csv

In [2]:
def json_to_csv(json_filename, csv_filename=None):
    try:
        if csv_filename is None:
            csv_filename = json_filename.replace('.json', '.csv')
        
        with open(json_filename, encoding='utf-8') as inputfile:
            df = pd.read_json(inputfile)
        
        df.to_csv(csv_filename, encoding='utf-8', index=False)
        
        return f"CSV file '{csv_filename}' created successfully."
    
    except FileNotFoundError:
        return f"Error: The file '{json_filename}' does not exist."
    except ValueError as e:
        return f"Error: {e}. Ensure the JSON structure is valid for conversion."
    except Exception as e:
        return f"An unexpected error occurred: {e}"


In [6]:
result = json_to_csv('data/Kovalsky_Wrapped2023.json')
print(result)

CSV file 'data/Kovalsky_Wrapped2023.csv' created successfully.


Spotify Web API install and set up

In [5]:
!pip install spotipy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\C25Michaela.Kovalsky\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up credentials (replace these with your actual client ID and secret)
CLIENT_ID = 'jujurazz'
CLIENT_SECRET = 'Redondo325'

# Authenticate with Spotify
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
